## Database

### Selecting databases

#### Four factors to consider when selecting databases
* structure of data
* query pattern
* amount or scale that you need to handle

#### Caching solution
* cosider to use a cache system (Redis) when you
  + call database very frequently
  + make a remote call to independent services with high latency
  
#### File storage solution
* consider to use Blob (binary large object) storage (s3) when you
  + need a data store for images, videos rahter than store data in database for queries
  + blob solutions are usually combined with a Content Delivery Network (CDN)
  + A CDN is a network of servers around the world that delivers content in different geographical locations with reduced latency
  
#### Elasticsearch with text search capability
* Consider to use Elesticsearch if you need to build a search functionality supporting fuzzy search
  + for example, search by movie, genre, actor, actress, director, etc
* Elasticsearch is not a database, which provide a guarantee that once stored, our data will not be lost unless we delete it
  + Elasticsearch offers no guarantee that our data will not be lost
  + we should never use search engines like Elasticsearch as our primary data source
* We can load the data to them from our primary database to reduce search latency and provide fuzzy and relevance-based text search

#### Time series database
* updated sequentially in an append-only format instead of random update
* have more bulk rads for a certain time ranges as opposed to random reads
* used for metric tacking systems such as grafana or cloud watch to watch how metric changes vs time
* calcualte how many people watched a video in the last 1 week, 10 days, 1 month, 1 year, and so on
* Examples of time series database is OpenTSDB and InfluxDB 

#### Data warehouse 
* a large database to dump all of the data available to perform analytics
* the systems are not used for regular transactions but offline reporting
* redshift, or s3/athena/EMR/spark

#### When to use SQL or NoSQL
* Data are structured and require transaction operations, use RDBMS such as mysql
* Data conain differnt data types and column names with complex query patterns, use document DB such as Mongo DB
* Data contain fixed column names with few query patterns, but volume increases very fast, use columnar DB, such as cassandra and dynamo
* you can combine RDBMS with other nosql DB
  + you need to process transaction in Aamzon, but also need to store a large amount of transaction history data. 
    + use mysql to store data that have not complete transactions. After transactions are completed, move those data to cassandra
  + use Mongo DB to store all the transacton/purchase history data, and query the data for a specific aggregation such as different product id and customer id in transactions of surgar in the last three months, and use those id to query mysql and cassandra to get more specific data  